In [1]:
DATA_NAME = 'coco-indoor' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_vertical_wavelet_indoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,4899.50,0.10,0.03,0.03,0.00,0.00,-0.00,-0.00
3,0.10,1238.64,-0.01,0.01,-0.00,0.00,-0.00,0.00
4,0.03,-0.01,263.90,0.00,0.00,-0.00,0.00,-0.00
5,0.03,0.01,0.00,51.53,0.00,0.00,0.00,-0.00
6,0.00,-0.00,0.00,0.00,9.18,0.00,0.00,0.00
7,0.00,0.00,-0.00,0.00,0.00,1.61,-0.00,0.00
8,-0.00,-0.00,0.00,0.00,0.00,-0.00,0.31,0.00
9,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.07


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,0.00004,0.00003,0.00005,0.00001,0.00000,-0.00000,-0.00002
3,0.00004,1.00000,-0.00002,0.00004,-0.00001,0.00002,-0.00005,0.00009
4,0.00003,-0.00002,1.00000,0.00002,0.00001,-0.00006,0.00007,-0.00006
5,0.00005,0.00004,0.00002,1.00000,0.00002,0.00002,0.00002,-0.00004
6,0.00001,-0.00001,0.00001,0.00002,1.00000,0.00002,0.00003,0.00004
7,0.00000,0.00002,-0.00006,0.00002,0.00002,1.00000,-0.00004,0.00001
8,-0.00000,-0.00005,0.00007,0.00002,0.00003,-0.00004,1.00000,0.00003
9,-0.00002,0.00009,-0.00006,-0.00004,0.00004,0.00001,0.00003,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.1527799140605043

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.00496025e+06 1.87865829e+05 8.29290181e+03 3.03657507e+02
 9.04863039e+00 2.48785371e-01 7.03745731e-03 1.09531724e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999230,-0.038450,-0.007618,-0.001479,-0.000264,-0.000046,-0.000009,-0.000002
1,0.038127,0.998546,-0.037454,-0.006930,-0.001234,-0.000216,-0.000042,-0.000008
2,0.008966,0.036815,0.998424,-0.040790,-0.006946,-0.001223,-0.000230,-0.000052
3,0.002090,0.008292,0.040120,0.998059,-0.046178,-0.007788,-0.001485,-0.000331
4,0.000464,0.001839,0.008637,0.045279,0.997042,-0.060411,-0.011078,-0.002451
5,0.000107,0.000424,0.002001,0.010221,0.058549,0.993150,-0.098429,-0.020687
6,0.000029,0.000117,0.000541,0.002781,0.015620,0.090744,0.971294,-0.219320
7,0.000014,0.000053,0.000253,0.001292,0.007243,0.041311,0.216274,0.975430


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0007696304419335664,
 0.001454097892145012,
 0.0015759333304385947,
 0.0019409381474841947,
 0.0029576931235335557,
 0.0068499090059173096,
 0.028705954234030107,
 0.02456952204762808]